In [ ]:
import pandas as pd
import pandas
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
dados = pd.read_csv("imdb-reviews-pt-br.csv")
dados.head()

In [ ]:
dados.head(5)

In [ ]:
dados.tail()

In [ ]:
dados.shape

In [ ]:
dados.info()

In [ ]:
print(dados.sentiment.value_counts())

In [ ]:
classificacao = dados["sentiment"].replace(["neg", "pos"], [0, 1])
dados["classificacao"] = classificacao
dados.head()

# Visualização de dados NLP

In [ ]:
ax = sns.distplot(dados["id"])

In [ ]:
ax = sns.countplot(dados["sentiment"])

In [ ]:
plt.figure(figsize=(10, 6))

plt.title("Classificação de sentimentos positivos e negativos")
plt.ylabel("Negativos")
plt.hist(dados["classificacao"]);
plt.legend(["Positivo", "Negativo"])
plt.show()

In [ ]:
ma = dados.corr()
ma

In [ ]:
corr = sns.heatmap(ma, annot = True, fmt = ".1f")

In [ ]:
ax = sns.pairplot(dados)

# Bag of words

**Criando representações da linguagem humana**

In [ ]:
print("Negativa \n")
print(dados.text_pt[189])

In [ ]:
print("Positivo \n", dados.text_pt[49002])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

texto = ["Hoje não estou feliz"]

vet = CountVectorizer(lowercase=False)
bag_of_words = vet.fit_transform(texto)

matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words, columns = vet.get_feature_names())
matriz_esparsa

In [ ]:
vetorizar = CountVectorizer(lowercase=False, max_features=50)
bag_of_words = vetorizar.fit_transform(dados.text_pt)
print(bag_of_words.shape)

# Visualizando os dados com WordCloud

In [ ]:
resenha_positiva = dados.query("sentiment == 'pos'")
resenha_positiva.head()

In [ ]:
resenha_negativa = dados.query("sentiment == 'neg'")
resenha_negativa

In [ ]:
%matplotlib inline
from wordcloud import WordCloud

todos_palavras = ' '.join([texto for texto in dados["text_pt"]])
nuvem_palavras = WordCloud(width = 800, height = 500, max_font_size = 110,
                          collocations = False).generate(todos_palavras)

plt.figure(figsize= (10, 7))
plt.imshow(nuvem_palavras, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
palavras_pos = ' '.join([texto for texto in resenha_positiva["text_pt"]])
nuvem_palavras = WordCloud(width=800, height=500, max_font_size=110,
                           collocations=False).generate(palavras_pos)

plt.figure(figsize=(10,7))
plt.imshow(nuvem_palavras, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
palavras_neg = ' '.join([texto for texto in resenha_negativa["text_pt"]])
nuvem_palavras = WordCloud(width=800, height=500, max_font_size=110,
                           collocations=False).generate(palavras_neg)

plt.figure(figsize=(10,7))
plt.imshow(nuvem_palavras, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
import nltk
from nltk import tokenize

frase = "Bem vindo ao mundo do PLN"
token_espaco = tokenize.WhitespaceTokenizer()
token_frase = token_espaco.tokenize(frase)
print(token_frase)

In [ ]:
todas_palavras = ' '.join([texto for texto in dados["text_pt"]])
frequencia = nltk.FreqDist(token_espaco.tokenize(todas_palavras))
df_frequencia = pd.DataFrame({"Palavra": list(frequencia.keys()),
                             "Frequência": list(frequencia.values())})

df_frequencia.head()

In [ ]:
df_frequencia = df_frequencia.nlargest(columns = "Frequência", n = 10)
df_frequencia

In [ ]:
plt.figure(figsize=(12,8))
ax = sns.barplot(data = df_frequencia, x= "Palavra", y = "Frequência", color = 'gray')
ax.set(ylabel = "Contagem")
plt.show()

In [ ]:
palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
frase_processada = list()

for opiniao in dados["text_pt"]:
    nova_frase = list()
    palavras_texto = token_espaco.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in palavras_irrelevantes:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
dados["tratamento_1"] = frase_processada
dados.head()

print("Frase text_pr\n")
print(dados["text_pt"][0])
print("\n Frase tratamento_1\n")
print(dados["tratamento_1"][0])

In [ ]:
def nuvem_palavras_neg(texto, coluna_texto):
    texto_negativo = texto.query("sentiment == 'neg'")
    todas_palavras = ' '.join([texto for texto in texto_negativo[coluna_texto]])
    nuvem_palavras = WordCloud(width = 800, height = 500,
                                max_font_size=110, collocations=False).generate(todas_palavras)
    plt.figure(figsize=(10,7))
    plt.imshow(nuvem_palavras, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    
nuvem_palavras_neg(dados, "tratamento_1")

In [ ]:
def nuvem_palavras_pos(texto, coluna_texto):
    texto_positivo = texto.query("sentiment == 'pos'")
    todas_palavras = ' '.join([texto for texto in texto_positivo[coluna_texto]])
    nuvem_palavras = WordCloud(width = 800, height = 500,
                                max_font_size=110, collocations=False).generate(todas_palavras)
    plt.figure(figsize=(10,7))
    plt.imshow(nuvem_palavras, interpolation='bilinear')
    plt.axis('off')
    plt.show()
    
nuvem_palavras_pos(dados, "tratamento_1")    

In [ ]:
def pareto(texto, coluna_texto, quantidade):
    todas_palavras = ' '.join([texto for texto in texto[coluna_texto]])
    frequencia = nltk.FreqDist(token_espaco.tokenize(todas_palavras))
    df_frequencia = pd.DataFrame({"Palavra": list(frequencia.keys()),
                                 "Frequência": list(frequencia.values())})
    df_frequencia = df_frequencia.nlargest(columns = "Frequência", n = quantidade)
    plt.figure(figsize=(12,8))
    ax = sns.barplot(data = df_frequencia, x= "Palavra", y = "Frequência", color = 'gray')
    ax.set(ylabel = "Contagem")
    plt.show()   

pareto(dados, "tratamento_1", 10)

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

def classificar_texto(texto, coluna_texto, coluna_classificacao):
    vetorizar = CountVectorizer(lowercase=False, max_features=50)
    bag_of_words = vetorizar.fit_transform(texto[coluna_texto])
    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                                 texto[coluna_classificacao],
                                                                 random_state = 42)
    
    regressao_logistica = LogisticRegression(solver="lbfgs")
    regressao_logistica.fit(treino, classe_treino)
    
    return regressao_logistica.score(teste, classe_teste)

acuracia_teste = classificar_texto(dados, "tratamento_1", "classificacao")
print(acuracia_teste)

# Modelo PLN 2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
regressao_logistica = LogisticRegression(solver = "lbfgs")

In [ ]:
from nltk import tokenize

frase = "Olá mundo!"
token_pontuacao = tokenize.WordPunctTokenizer()
token_frase = token_pontuacao.tokenize(frase)
token_frase

In [ ]:
from string import punctuation

pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)

pontuacao_stopwords = pontuacao + palavras_irrelevantes

frase_processada = list()
for opiniao in dados["tratamento_1"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in pontuacao_stopwords:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
dados["tratamento_2"] = frase_processada
dados.head()

In [ ]:
dados["tratamento_1"][0]

In [ ]:
dados["tratamento_2"][0]

In [ ]:
pareto(dados, "tratamento_2", 10)

In [ ]:
import unidecode
acentos = "Ótimo péssimo não é tão"
teste = unidecode.unidecode(acentos)
teste

In [ ]:
sem_acentos = [unidecode.unidecode(texto) for texto in dados["tratamento_2"]]
sem_acentos

In [ ]:
stopwords_sem_acento = [unidecode.unidecode(texto) for texto in pontuacao_stopwords]
stopwords_sem_acento

In [ ]:
dados["tratamento_3"] = sem_acentos

frase_processada = list()
for opiniao in dados["tratamento_3"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
dados["tratamento_3"] = frase_processada

acuracia_tratamento3 = classificar_texto(dados, "tratamento_3", "classificacao")
print(acuracia_tratamento3)

In [ ]:
acuracia_teste

In [ ]:
frase_processada = list()
for opiniao in dados["tratamento_3"]:
    nova_frase = list()
    opiniao = opiniao.lower()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(palavra)
    frase_processada.append(' '.join(nova_frase))
    
dados["tratamento_4"] = frase_processada
dados.head()

In [ ]:
dados["text_pt"][0]

In [ ]:
dados["tratamento_4"][0]

In [ ]:
acuracia_tratamento4 = classificar_texto(dados, "tratamento_4", "classificacao")
print(acuracia_tratamento4)
print(acuracia_tratamento3)

In [ ]:
nuvem_palavras_neg(dados, "tratamento_4")

In [ ]:
nuvem_palavras_pos(dados, "tratamento_4")

In [ ]:
pareto(dados, "tratamento_4", 10)

In [ ]:
import nltk
nltk.download('rslp')

In [ ]:
stemmer = nltk.RSLPStemmer()
st = stemmer.stem("correria")
st

In [ ]:
frase_processada = list()
for opiniao in dados["tratamento_4"]:
    nova_frase = list()
    palavras_texto = token_pontuacao.tokenize(opiniao)
    for palavra in palavras_texto:
        if palavra not in stopwords_sem_acento:
            nova_frase.append(stemmer.stem(palavra))
    frase_processada.append(' '.join(nova_frase))
    
dados["tratamento_5"] = frase_processada
dados.head()

In [ ]:
acuracia_tratamento5 = classificar_texto(dados, "tratamento_5", "classificacao")
print(acuracia_tratamento5)
print(acuracia_tratamento4)

In [ ]:
nuvem_palavras_neg(dados, "tratamento_5")

In [ ]:
nuvem_palavras_pos(dados,"tratamento_5")

In [ ]:
pareto(dados, "tratamento_5", 10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

frases = ["Assiti um filme ótimo", "Assiti um filme péssimo"]

tfidf = TfidfVectorizer(lowercase = False, max_features = 50)
caracteristicas = tfidf.fit_transform(frases)

pd.DataFrame(
caracteristicas.todense(),
columns = tfidf.get_feature_names()
)

In [ ]:
tfidf_bruto = tfidf.fit_transform(dados["text_pt"])

In [ ]:
tfidf_tratados = tfidf.fit_transform(dados["tratamento_5"])

# Treino e teste 1

In [ ]:
#Treino e teste

treino, teste, classe_treino, classe_teste = train_test_split(tfidf_tratados,
                                                              dados["classificacao"],
                                                              random_state = 42)

In [ ]:
treino.shape

In [ ]:
classe_treino.shape

In [ ]:
regressao_logistica.fit(treino, classe_treino)
acuracia_tfidf_tratados = regressao_logistica.score(teste, classe_teste)
print(acuracia_tfidf_tratados)

In [ ]:
print(acuracia_tratamento5)

In [ ]:
from nltk import ngrams

frase = "Assisti um ótimo filme."
frase_separada = token_espaco.tokenize(frase)
frase_separada

In [ ]:
pares = ngrams(frase_separada, 2)
pares
list(pares)

# Treino e teste 2

In [ ]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
tfidf = TfidfVectorizer(lowercase=False)
vetor_tfidf = tfidf.fit_transform(dados["tratamento_5"])
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf,
                                                              dados["classificacao"],
                                                              random_state = 42)

In [ ]:
treino.shape

In [ ]:
classe_treino.shape

# Modelo de regressão logistica  (Melhorado)


In [ ]:
regressao_logistica.fit(treino, classe_treino)
reg_log = regressao_logistica.score(teste, classe_teste)
print(reg_log)

In [ ]:
pesos = pd.DataFrame(
    regressao_logistica.coef_[0].T,
    index = tfidf.get_feature_names()
)

pesos.nlargest(50,0)

In [ ]:
pesos.nsmallest(10,0)